<a href="https://colab.research.google.com/github/menna161/Mining-API-Usage-Patterns/blob/main/FromCodetoPattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install astor

: 

In [ ]:
import ast
import astor

# Input Examples

In [ ]:
ex1 = '''
x = txt.split()
print(x)
'''


ex2 ='''
for line in content.split('.'):
    print(line)
'''

ex3 = '''
words = dir.split('/')
return words[0]
'''

ex4 = '''
lines = k.split(".")
first = lines.pop()
'''

ex5 = '''
parts  = k.split('.')
line = lines.pop()
'''

ex6 = '''
parts = package_name.split('.')
name = parts.pop()
'''

# Convert Code to AST

In [ ]:
tree1 = ast.parse(ex1)
tree2 = ast.parse(ex2)
tree3 = ast.parse(ex3)
tree4 = ast.parse(ex4)
tree5 = ast.parse(ex5)
tree6 = ast.parse(ex6)


dumpp1 = ast.dump(tree1)
dumpp2 = ast.dump(tree2)
dumpp3 = ast.dump(tree3)
dumpp4 = ast.dump(tree4)
dumpp5 = ast.dump(tree5)
dumpp6 = ast.dump(tree6)


print("ex1", dumpp1)
print("ex2", dumpp2)
print("ex3", dumpp3)
print("ex4", dumpp4)
print("ex5", dumpp5)
print("ex6", dumpp6)


# AST To Gspan Format

In [ ]:
code_snippets = [tree1, tree2, tree3, tree4, tree5, tree6]

file = open("input.txt", "w")
i=0


# print(code_snippets)
for snippet_tree in code_snippets:
  file_write= []
  v_n = 0;
  parent_n = -1;
  file.write(f't # {i}\n')
  file.write(f'v 0 Module\n')
  i+=1
  # print("tree source code")
  # print(astor.to_source(snippet_tree))


  for node in ast.walk(snippet_tree):
      n = str(node).split()[0][6:]
      parent_n += 1
      children = []
      e_n = 0
      # print("Node", parent_n, n)

      for x in ast.iter_child_nodes(node): 
          child = type(x).__name__
          # print("childdd", child)
          name = x._fields
          # print("nameee", name)
          for item,val in ast.iter_fields(x):
            if(item in ("name","id","attr")):
              child = str(child)+ "#"+item+"="+val
          v_n +=1
          # print(f'v {v_n} {child}\n')
          # print("---Children",child )
          # print("list children ",list(ast.iter_child_nodes(node)))
          file_write.append('v '+str(v_n)+' '+str(child)+'\n')
          # print(file_write)
          children.append([child,v_n])



      # print("children ", children)
      # print("list ",list(ast.iter_fields(node)))
      edges_list = list(ast.iter_fields(node))

      for edge in list(ast.iter_fields(node)):
      #   print("edge ", edge)
        
        if not(edge[0] in ("name","id","attr","str")) and len(children) > 0:
          # print("edge[0] ", edge[0])
          # print("edge[1] ", edge[1])

          if isinstance(edge[1], list):
            if(len(edge[1]) == 1 ):
                # print(f'e {parent_n} {children[e_n][1]} {edge[0]}\n')
                # print(f'e {parent_n} {children[e_n][1]} {edge[0]}\n')
                file_write.append('e '+str(parent_n)+' '+str(children[e_n][1])+' '+str(edge[0])+'\n')
                e_n+=1
            else:
              edge_name_n = 0;
              for node in edge[1]:
                # print("node ",node)
                # print(f'e {parent_n} {children[e_n][1]} {edge[0]+str(edge_name_n)}\n')
                # file.write(f'e {parent_n} {children[e_n][1]} {edge[0]+str(edge_name_n)}\n')
                file_write.append('e '+str(parent_n)+' '+str(children[e_n][1])+' '+str(edge[0]+str(edge_name_n))+'\n')
                e_n+=1
                edge_name_n+=1
          else:
            # print(f'e {parent_n} {children[e_n][1]} {edge[0]}\n')
            # file.write(f'e {parent_n} {children[e_n][1]} {edge[0]}\n')
            file_write.append('e '+str(parent_n)+' '+str(children[e_n][1])+' '+str(edge[0])+'\n')
            # print("e_n ", e_n)
            e_n+=1

  # print("not sorted")
  # print(file_write)
  file_write.sort()
  # print("sorted")
  # print(file_write)
  for y in reversed(file_write):
    file.write(y)
        

file.write("t # -1\n")
file.close()



# Minning Common Patter with Gspan

In [ ]:
!pip install gspan-mining

In [ ]:
!python -m gspan_mining -s 3 ./input.txt #max vertices , support 50%

In [ ]:
# t # 27
# v 0 Assign
# v 1 Module
# v 2 Assign
# v 3 Call
# v 4 Attribute#attr=pop
# v 5 Load
# v 6 Call
# v 7 Str
# v 8 Attribute#attr=split
# v 9 Load
# e 0 1 body0
# e 0 6 value
# e 1 2 body1
# e 2 3 value
# e 3 4 func
# e 4 5 ctx
# e 6 7 args
# e 6 8 func
# e 8 9 ctx

# Support: 3

In [ ]:
from gspan_mining.config import parser
from gspan_mining.main import main

In [ ]:
%pylab inline

In [ ]:
!python -m gspan_mining -h

In [ ]:
args_str = '-s 3  -d False -l 3 -p True  -w True input.txt'
#args_str = '-s 3 -d True -l 5 -p True -w True final.txt'
FLAGS, _ = parser.parse_known_args(args=args_str.split())

In [ ]:
gs = main(FLAGS)